In [28]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.ensemble import IsolationForest, AdaBoostClassifier, BaggingClassifier, BaseEnsemble, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from catboost import CatBoostClassifier
import eli5
import shap
from sklearn import model_selection
from imblearn.under_sampling import TomekLinks
import umap

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [2]:
train = pd.read_csv('../data/CAX_MortgageModeling_Train.csv')
train_len = train.shape[0]

In [4]:
data2 = pd.read_csv('../data/data_full_final.csv')

In [11]:
train = data2.iloc[:train_len]
test = data2.iloc[train_len:]

In [12]:
train2 = pd.concat([train[train.RESULT == 0].sample(frac=.3,random_state=123),train[train.RESULT == 1]],axis=0).sample(frac=1,random_state=123)

In [13]:
train2.head(10)

,PROPERTY VALUE,MORTGAGE PAYMENT,GDS,LTV,TDS,MORTGAGE AMOUNT,RATE,AGE RANGE,INCOME,CREDIT SCORE,RESULT,APPRAISED PROPERTY VALUE,CHANGE IN PROPERTY VALUE,ANNUALIZED HOUSING EXPENSE,MORTGAGE RATIO,OTHER EXPENSE,RENT INCOME,ACTUAL MORTGAGE PAYMENT,TOTAL INTEREST,MORTGAGE AMOUNT PER INCOME,MORTGAGE PAYMENT PER INCOME,PROPERTY VALUE PER INCOME,TOTAL INTEREST PER INCOME,OTHER EXPENSE PER INCOME,ANNUALIZED HOUSING EXPENSE PER INCOME,ACTUAL MORTGAGE PAYMENT BY MORTGAGE,PROPERTY VALUE MORTGAGE PURPOSE MEAN,PROPERTY VALUE MORTGAGE PURPOSE VAR,PROPERTY VALUE MORTGAGE PURPOSE Z SCORE,MORTGAGE PAYMENT MORTGAGE PURPOSE MEAN,MORTGAGE PAYMENT MORTGAGE PURPOSE VAR,MORTGAGE PAYMENT MORTGAGE PURPOSE Z SCORE,GDS MORTGAGE PURPOSE MEAN,GDS MORTGAGE PURPOSE VAR,GDS MORTGAGE PURPOSE Z SCORE,LTV MORTGAGE PURPOSE MEAN,LTV MORTGAGE PURPOSE VAR,LTV MORTGAGE PURPOSE Z SCORE,TDS MORTGAGE PURPOSE MEAN,TDS MORTGAGE PURPOSE VAR,TDS MORTGAGE PURPOSE Z SCORE,MORTGAGE AMOUNT MORTGAGE PURPOSE MEAN,MORTGAGE AMOUNT MORTGAGE PURPOSE VAR,MORTGAGE AMOUNT MORTGAGE PURPOSE Z SCORE,INCOME MORTGAGE PURPOSE MEAN,INCOME MORTGAGE PURPOSE VAR,INCOME MORTGAGE PURPOSE Z SCORE,CREDIT SCORE MORTGAGE PURPOSE MEAN,CREDIT SCORE MORTGAGE PURPOSE VAR,CREDIT SCORE MORTGAGE PURPOSE Z SCORE,APPRAISED PROPERTY VALUE MORTGAGE PURPOSE MEAN,APPRAISED PROPERTY VALUE MORTGAGE PURPOSE VAR,APPRAISED PROPERTY VALUE MORTGAGE PURPOSE Z SCORE,CHANGE IN PROPERTY VALUE MORTGAGE PURPOSE MEAN,CHANGE IN PROPERTY VALUE MORTGAGE PURPOSE VAR,CHANGE IN PROPERTY VALUE MORTGAGE PURPOSE Z SCORE,ANNUALIZED HOUSING EXPENSE MORTGAGE PURPOSE MEAN,ANNUALIZED HOUSING EXPENSE MORTGAGE PURPOSE VAR,ANNUALIZED HOUSING EXPENSE MORTGAGE PURPOSE Z SCORE,MORTGAGE RATIO MORTGAGE PURPOSE MEAN,MORTGAGE RATIO MORTGAGE PURPOSE VAR,MORTGAGE RATIO MORTGAGE PURPOSE Z SCORE,OTHER EXPENSE MORTGAGE PURPOSE MEAN,OTHER EXPENSE MORTGAGE PURPOSE VAR,OTHER EXPENSE MORTGAGE PURPOSE Z SCORE,ACTUAL MORTGAGE PAYMENT MORTGAGE PURPOSE MEAN,ACTUAL MORTGAGE PAYMENT MORTGAGE PURPOSE VAR,ACTUAL MORTGAGE PAYMENT MORTGAGE PURPOSE Z SCORE,TOTAL INTEREST MORTGAGE PURPOSE MEAN,TOTAL INTEREST MORTGAGE PURPOSE VAR,TOTAL INTEREST MORTGAGE PURPOSE Z SCORE,MORTGAGE AMOUNT PER INCOME MORTGAGE PURPOSE MEAN,MORTGAGE AMOUNT PER INCOME MORTGAGE PURPOSE VAR,MORTGAGE AMOUNT PER INCOME MORTGAGE PURPOSE Z SCORE,MORTGAGE PAYMENT PER INCOME MORTGAGE PURPOSE MEAN,MORTGAGE PAYMENT PER INCOME MORTGAGE PURPOSE VAR,MORTGAGE PAYMENT PER INCOME MORTGAGE PURPOSE Z SCORE,PROPERTY VALUE PER INCOME MORTGAGE PURPOSE MEAN,PROPERTY VALUE PER INCOME MORTGAGE PURPOSE VAR,PROPERTY VALUE PER INCOME MORTGAGE PURPOSE Z SCORE,TOTAL INTEREST PER INCOME MORTGAGE PURPOSE MEAN,TOTAL INTEREST PER INCOME MORTGAGE PURPOSE VAR,TOTAL INTEREST PER INCOME MORTGAGE PURPOSE Z SCORE,OTHER EXPENSE PER INCOME MORTGAGE PURPOSE MEAN,OTHER EXPENSE PER INCOME MORTGAGE PURPOSE VAR,OTHER EXPENSE PER INCOME MORTGAGE PURPOSE Z SCORE,ANNUALIZED HOUSING EXPENSE PER INCOME MORTGAGE PURPOSE MEAN,ANNUALIZED HOUSING EXPENSE PER INCOME MORTGAGE PURPOSE VAR,ANNUALIZED HOUSING EXPENSE PER INCOME MORTGAGE PURPOSE Z SCORE,ACTUAL MORTGAGE PAYMENT BY MORTGAGE MORTGAGE PURPOSE MEAN,ACTUAL MORTGAGE PAYMENT BY MORTGAGE MORTGAGE PURPOSE VAR,ACTUAL MORTGAGE PAYMENT BY MORTGAGE MORTGAGE PURPOSE Z SCORE,PROPERTY VALUE PAYMENT FREQUENCY MEAN,PROPERTY VALUE PAYMENT FREQUENCY VAR,PROPERTY VALUE PAYMENT FREQUENCY Z SCORE,MORTGAGE PAYMENT PAYMENT FREQUENCY MEAN,MORTGAGE PAYMENT PAYMENT FREQUENCY VAR,MORTGAGE PAYMENT PAYMENT FREQUENCY Z SCORE,GDS PAYMENT FREQUENCY MEAN,GDS PAYMENT FREQUENCY VAR,GDS PAYMENT FREQUENCY Z SCORE,LTV PAYMENT FREQUENCY MEAN,LTV PAYMENT FREQUENCY VAR,LTV PAYMENT FREQUENCY Z SCORE,TDS PAYMENT FREQUENCY MEAN,TDS PAYMENT FREQUENCY VAR,TDS PAYMENT FREQUENCY Z SCORE,MORTGAGE AMOUNT PAYMENT FREQUENCY MEAN,MORTGAGE AMOUNT PAYMENT FREQUENCY VAR,MORTGAGE AMOUNT PAYMENT FREQUENCY Z SCORE,INCOME PAYMENT FREQUENCY MEAN,INCOME PAYMENT FREQUENCY VAR,INCOME PAYMENT FREQUENCY Z SCORE,CREDIT SCORE PAYMENT FREQUENCY MEAN,CREDIT SCOR

In [9]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def predict_proba(self, x):
        return self.clf.predict_proba(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [10]:
SEED = 123 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(n_splits= NFOLDS, random_state=SEED)

def get_oof(clf, x_train, y_train, x_test):
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict_proba(x_te)[:,1]
        oof_test_skf[i, :] = clf.predict_proba(x_test)[:,1]

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [14]:
rf_params = {'n_estimators': 325, 'max_depth': 10, 'random_state': 123}
gbm_params = {'n_estimators': 475, 'max_depth': 7, 'max_features': 0.9, 'random_state': 123}
xgb_params = {'max_depth': 7, 'gamma': '0.479', 'colsample_bytree': '0.472', 'n_estimators': 375, 'random_state':123}
lgb_params = {'max_depth': 4, 'colsample_bytree': '0.418', 'n_estimators': 250, 'random_state':123}
et_params = {'n_estimators': 500, 'max_depth': 10, 'random_state': 123}

In [16]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
gbm = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gbm_params)
xgb = SklearnHelper(clf=XGBClassifier, seed=SEED, params=xgb_params)
lgb = SklearnHelper(clf=LGBMClassifier, seed=SEED, params=lgb_params)

In [17]:
traincols = list(data2.columns)
traincols.remove('RESULT')
x_train = train2[traincols].values
y_train = train2.RESULT.values
x_test = test[traincols].values

In [18]:
rf_oof_train, rf_oof_test = get_oof(rf, x_train, y_train, x_test)
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test)
gbm_oof_train, gbm_oof_test = get_oof(gbm, x_train, y_train, x_test)
xgb_oof_train, xgb_oof_test = get_oof(xgb, x_train, y_train, x_test)
lgb_oof_train, lgb_oof_test = get_oof(lgb, x_train, y_train, x_test)

In [19]:
train_pred = pd.DataFrame()
train_pred['rf'] = rf_oof_train.reshape(1,-1).tolist()[0]
train_pred['et'] = et_oof_train.reshape(1,-1).tolist()[0]
train_pred['gbm'] = gbm_oof_train.reshape(1,-1).tolist()[0]
train_pred['xgb'] = xgb_oof_train.reshape(1,-1).tolist()[0]
train_pred['lgb'] = lgb_oof_train.reshape(1,-1).tolist()[0]
train_pred['actual'] = train2.RESULT.values

In [20]:
test_pred = pd.DataFrame()
test_pred['rf'] = rf_oof_test.reshape(1,-1).tolist()[0]
test_pred['et'] = et_oof_test.reshape(1,-1).tolist()[0]
test_pred['gbm'] = gbm_oof_test.reshape(1,-1).tolist()[0]
test_pred['xgb'] = xgb_oof_test.reshape(1,-1).tolist()[0]
test_pred['lgb'] = lgb_oof_test.reshape(1,-1).tolist()[0]

In [21]:
train_pred.head(10)

,rf,et,gbm,xgb,lgb,actual
0,0.731990,0.644671,0.716872,0.613996,0.510704,0.0
1,0.545058,0.477933,0.710986,0.620941,0.538286,1.0
2,0.601606,0.569789,0.522910,0.703614,0.522205,0.0
3,0.276842,0.354266,0.060978,0.135826,0.228923,1.0
4,0.501551,0.577039,0.423388,0.344085,0.347665,0.0
5,0.412766,0.479354,0.134339,0.188750,0.274171,0.0
6,0.420403,0.441276,0.795476,0.655472,0.581950,1.0
7,0.427251,0.424242,0.324943,0.224284,0.377301,0.0
8,0.436928,0.396033,0.545638,0.690225,0.464660,0.0
9,0.525043,0.551541,0.281134,0.262807,0.331574,0.0


In [24]:
for col in ['rf','et','gbm','xgb','lgb']:
    print (col, f1_score(train_pred.actual,np.round(train_pred[col].values)))

rf 0.6135844127571568
et 0.591423528190219
gbm 0.633351264120495
xgb 0.6321126152850439
lgb 0.639373013263181


In [29]:
rd = RidgeClassifier()
cross_val_score(rd,train_pred.values[:,:5],train_pred.actual.values,scoring='f1',cv=KFold(5))

array([0.64549685, 0.63126362, 0.64046179, 0.63973799, 0.64357786])

## Meta modelling

In [30]:
data = pd.read_csv('../data/data_full_categorical_1705.csv')
train_meta = data.iloc[:train_len]
test_meta = data.iloc[train_len:]

In [31]:
cnt_cols_meta = []
cat_cols_meta = []
for col in data.columns:
    if data[col].dtype != object and col != 'RESULT' and 'ordinal' not in col and data[col].nunique() > 10:
        cnt_cols_meta.append(col)
    elif 'Z SCORE' not in col:
        cat_cols_meta.append(col)

In [32]:
cat_cols_meta.remove('Unique_ID')
cat_cols_meta.remove('RESULT')

In [33]:
train2_meta = pd.concat([train_meta[train_meta.RESULT == 0].sample(frac=.3,random_state=123),train_meta[train_meta.RESULT == 1]],axis=0).sample(frac=1,random_state=123)

In [34]:
train2_meta.head(10)

,Unique_ID,PROPERTY VALUE,MORTGAGE PAYMENT,GDS,LTV,TDS,AMORTIZATION,MORTGAGE AMOUNT,RATE,MORTGAGE PURPOSE,PAYMENT FREQUENCY,PROPERTY TYPE,TERM,FSA,AGE RANGE,GENDER,INCOME,INCOME TYPE,NAICS CODE,CREDIT SCORE,RESULT,APPRAISED PROPERTY VALUE,CHANGE IN PROPERTY VALUE,ANNUALIZED HOUSING EXPENSE,MORTGAGE RATIO,OTHER EXPENSE,RENT INCOME,PAYMENT FREQUENCY IN MONTHS,ACTUAL MORTGAGE PAYMENT,TOTAL INTEREST,MORTGAGE AMOUNT PER INCOME,MORTGAGE PAYMENT PER INCOME,PROPERTY VALUE PER INCOME,TOTAL INTEREST PER INCOME,OTHER EXPENSE PER INCOME,ANNUALIZED HOUSING EXPENSE PER INCOME,ACTUAL MORTGAGE PAYMENT BY MORTGAGE,Is unemployed,Is recently married,Is married,Is established,Is about to retire,Is retired,REPAYMENT AGE,AGE RANGE_MORTGAGE PURPOSE_PROPERTY TYPE,GENDER_MORTGAGE PURPOSE_PROPERTY TYPE,FSA_MORTGAGE PURPOSE_PROPERTY TYPE,NAICS CODE_MORTGAGE PURPOSE_PROPERTY TYPE,PROPERTY VALUE_ordinal,MORTGAGE PAYMENT_ordinal,GDS_ordinal,LTV_ordinal,TDS_ordinal,MORTGAGE AMOUNT_ordinal,RATE_ordinal,AGE RANGE_ordinal,INCOME_ordinal,INCOME TYPE_ordinal,CREDIT SCORE_ordinal,APPRAISED PROPERTY VALUE_ordinal,CHANGE IN PROPERTY VALUE_ordinal,ANNUALIZED HOUSING EXPENSE_ordinal,MORTGAGE RATIO_ordinal,OTHER EXPENSE_ordinal,RENT INCOME_ordinal,ACTUAL MORTGAGE PAYMENT_ordinal,TOTAL INTEREST_ordinal,MORTGAGE AMOUNT PER INCOME_ordinal,MORTGAGE PAYMENT PER INCOME_ordinal,PROPERTY VALUE PER INCOME_ordinal,TOTAL INTEREST PER INCOME_ordinal,OTHER EXPENSE PER INCOME_ordinal,ANNUALIZED HOUSING EXPENSE PER INCOME_ordinal,ACTUAL MORTGAGE PAYMENT BY MORTGAGE_ordinal,REPAYMENT AGE_ordinal
8361,CAX_Train_8362,780000,4139,0.4286,0.7813,0.4324,360,615000.0,0.0575,Refinance,Monthly,Single Detached,12,L6,34,Male,109000,8,56,565,0.0,787149.622424,0.009166,46717.4,0.788462,414.2,0.0,1,3.451700e+06,2.836700e+06,5.642202,0.455670,7.155963,26.024775,0.0038,0.4286,5.612521,0,1,0,0,0,0,64.0,34_Refinance_Single Detached,Male_Refinance_Single Detached,L6_Refinance_Single Detached,56_Refinance_Single Detached,8,9,9,6,8,8,9,1,5,7,1,8,7,8,5,2,4,9,9,8,9,7,9,2,9,9,1
36510,CAX_Train_36511,250000,760,0.0791,0.4800,0.2053,300,120000.0,0.0600,Refinance,Monthly,Single Detached,36,K7,34,Female,159000,3,53,599,1.0,250000.000000,0.000000,12576.9,0.480000,20065.8,0.0,1,5.378027e+05,4.178027e+05,0.754717,0.057358,1.572327,2.627690,0.1262,0.0791,4.481689,0,1,0,0,0,0,59.0,34_Refinance_Single Detached,Female_Refinance_Single Detached,K7_Refinance_Single Detached,53_Refinance_Single Detached,0,0,0,0,0,0,9,1,7,4,1,0,4,1,0,7,4,0,0,0,0,0,0,7,0,7,0
25587,CAX_Train_25588,950000,4333,0.3104,0.8000,0.3907,360,760000.0,0.0475,Refinance,Monthly,Single Detached,36,L5,39,Female,187000,8,54,695,0.0,950000.000000,0.000000,58044.8,0.800000,15016.1,0.0,1,3.159972e+06,2.399972e+06,4.064171,0.278053,5.080214,12.834075,0.0803,0.3104,4.157858,0,0,1,1,0,0,69.0,39_Refinance_Single Detached,Female_Refinance_Single Detached,L5_Refinance_Single Detached,54_Refinance_Single Detached,8,9,5,8,5,9,5,2,8,7,5,8,4,9,7,7,4,9,9,5,5,4,6,5,5,6,2
3746,CAX_Train_3747,718000,2942,0.2300,0.8000,0.2890,360,574400.0,0.0350,Purchase,Monthly,Single Detached,12,L4,39,Female,160000,8,72,673,1.0,718000.000000,0.000000,36800.0,0.800000,9440.0,0.0,1,1.641435e+06,1.067035e+06,3.590000,0.220650,4.487500,6.668968,0.0590,0.2300,2.857651,0,0,1,1,0,0,69.0,39_Purchase_Single Detached,Female_Purchase_Single Detached,L4_Purchase_Single Detached,72_Purchase_Single Detached,7,7,3,8,1,8,0,2,8,7,4,7,4,7,7,6,4,6,5,4,3,3,1,4,3,0,2
13755,CAX_Train_13756,375000,1468,0.1279,0.6500,0.3803,300,203450.0,0.0550,Refinance,Monthly,Single Detached,12,A1,34,Male,126000,2,23,653,0.0,313000.000000,-0.165333,16115.4,0.542533,31802.4,0.0,1,8.046604e+05,6.012104e+05,1.614683,0.139810,2.976190,4.771511,0.2524,0.1279,3.955077,0,1,0,0,0,0,59.0,34_Refinance_Single Detached,Male_Refinance_Single Detached,A1_Refinance_Single Detached,23_Refinance_Single Detached,3,2,1,2,4,1,9,1,6,2,3,1,0,2,0,8,4,1,1,0,1,1,0,8,1,5,0
41356,CAX_Train_41357,138000,773,0.2312,0.7247,0.2364,300,99000.0,0.0500,Refin

In [35]:
for col in ['rf','et','gbm','xgb','lgb']:
    train2_meta[col] = train_pred[col]
    test_meta[col] = test_pred[col]
    cnt_cols_meta.append(col)

C:\Users\asengup6\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
for train_index2_meta, test_index2_meta in kf.split(train2_meta):
    break

In [37]:
meta_cb_model = CatBoostClassifier(iterations=5000,learning_rate=0.15, loss_function= 'Logloss', eval_metric='F1',use_best_model=True,random_seed=42)

In [39]:
meta_cb_model.fit(train2_meta[cat_cols_meta+cnt_cols_meta].iloc[train_index2_meta],train2_meta.RESULT.iloc[train_index2_meta],cat_features=np.arange(len(cat_cols_meta)),eval_set=(train2_meta[cat_cols_meta+cnt_cols_meta].iloc[test_index2_meta],train2_meta.RESULT.iloc[test_index2_meta]))

0:	learn: 0.5296094	test: 0.5218924	best: 0.5218924 (0)	total: 533ms	remaining: 44m 24s
1:	learn: 0.5965920	test: 0.6054809	best: 0.6054809 (1)	total: 1.06s	remaining: 44m 19s
2:	learn: 0.5908342	test: 0.6048128	best: 0.6054809 (1)	total: 1.63s	remaining: 45m 11s
3:	learn: 0.5856538	test: 0.5974588	best: 0.6054809 (1)	total: 2.16s	remaining: 44m 57s
4:	learn: 0.5860014	test: 0.6044278	best: 0.6054809 (1)	total: 2.48s	remaining: 41m 14s
5:	learn: 0.5926332	test: 0.6072468	best: 0.6072468 (5)	total: 2.94s	remaining: 40m 48s
6:	learn: 0.5923645	test: 0.6084893	best: 0.6084893 (6)	total: 3.48s	remaining: 41m 18s
7:	learn: 0.5936989	test: 0.6077145	best: 0.6084893 (6)	total: 3.89s	remaining: 40m 29s
8:	learn: 0.5941342	test: 0.6039341	best: 0.6084893 (6)	total: 4.35s	remaining: 40m 12s
9:	learn: 0.6015048	test: 0.6060283	best: 0.6084893 (6)	total: 4.86s	remaining: 40m 25s
10:	learn: 0.6052360	test: 0.6078589	best: 0.6084893 (6)	total: 5.36s	remaining: 40m 29s
11:	learn: 0.6052704	test: 0.60

92:	learn: 0.6482663	test: 0.6302268	best: 0.6302268 (92)	total: 34.7s	remaining: 30m 29s
93:	learn: 0.6488285	test: 0.6294777	best: 0.6302268 (92)	total: 35.2s	remaining: 30m 38s
94:	learn: 0.6496913	test: 0.6285558	best: 0.6302268 (92)	total: 35.6s	remaining: 30m 39s
95:	learn: 0.6495525	test: 0.6285558	best: 0.6302268 (92)	total: 36s	remaining: 30m 41s
96:	learn: 0.6490461	test: 0.6292563	best: 0.6302268 (92)	total: 36.6s	remaining: 30m 47s
97:	learn: 0.6495976	test: 0.6292563	best: 0.6302268 (92)	total: 37.2s	remaining: 30m 59s
98:	learn: 0.6510435	test: 0.6297105	best: 0.6302268 (92)	total: 37.7s	remaining: 31m 5s
99:	learn: 0.6501909	test: 0.6299127	best: 0.6302268 (92)	total: 38.3s	remaining: 31m 18s
100:	learn: 0.6502360	test: 0.6299127	best: 0.6302268 (92)	total: 38.8s	remaining: 31m 23s
101:	learn: 0.6501423	test: 0.6299127	best: 0.6302268 (92)	total: 39.4s	remaining: 31m 29s
102:	learn: 0.6511725	test: 0.6300846	best: 0.6302268 (92)	total: 39.8s	remaining: 31m 34s
103:	learn

182:	learn: 0.6698664	test: 0.6353005	best: 0.6360674 (171)	total: 1m 11s	remaining: 31m 13s
183:	learn: 0.6697739	test: 0.6353005	best: 0.6360674 (171)	total: 1m 11s	remaining: 31m 11s
184:	learn: 0.6698658	test: 0.6353005	best: 0.6360674 (171)	total: 1m 11s	remaining: 31m 11s
185:	learn: 0.6712529	test: 0.6351020	best: 0.6360674 (171)	total: 1m 12s	remaining: 31m 10s
186:	learn: 0.6711148	test: 0.6351020	best: 0.6360674 (171)	total: 1m 12s	remaining: 31m 11s
187:	learn: 0.6714882	test: 0.6355955	best: 0.6360674 (171)	total: 1m 13s	remaining: 31m 11s
188:	learn: 0.6726426	test: 0.6363636	best: 0.6363636 (188)	total: 1m 13s	remaining: 31m 11s
189:	learn: 0.6732880	test: 0.6354733	best: 0.6363636 (188)	total: 1m 13s	remaining: 31m 10s
190:	learn: 0.6735190	test: 0.6354733	best: 0.6363636 (188)	total: 1m 14s	remaining: 31m 9s
191:	learn: 0.6725541	test: 0.6354479	best: 0.6363636 (188)	total: 1m 14s	remaining: 31m 11s
192:	learn: 0.6726464	test: 0.6358193	best: 0.6363636 (188)	total: 1m 1

271:	learn: 0.6865982	test: 0.6370087	best: 0.6398040 (228)	total: 1m 47s	remaining: 31m 9s
272:	learn: 0.6863725	test: 0.6377523	best: 0.6398040 (228)	total: 1m 47s	remaining: 31m 8s
273:	learn: 0.6863248	test: 0.6366612	best: 0.6398040 (228)	total: 1m 48s	remaining: 31m 7s
274:	learn: 0.6871532	test: 0.6379733	best: 0.6398040 (228)	total: 1m 48s	remaining: 31m 7s
275:	learn: 0.6871532	test: 0.6379733	best: 0.6398040 (228)	total: 1m 49s	remaining: 31m 8s
276:	learn: 0.6877037	test: 0.6366857	best: 0.6398040 (228)	total: 1m 49s	remaining: 31m 10s
277:	learn: 0.6880677	test: 0.6366612	best: 0.6398040 (228)	total: 1m 50s	remaining: 31m 11s
278:	learn: 0.6885564	test: 0.6383442	best: 0.6398040 (228)	total: 1m 50s	remaining: 31m 11s
279:	learn: 0.6892538	test: 0.6383442	best: 0.6398040 (228)	total: 1m 50s	remaining: 31m 10s
280:	learn: 0.6890244	test: 0.6385181	best: 0.6398040 (228)	total: 1m 51s	remaining: 31m 9s
281:	learn: 0.6892632	test: 0.6385181	best: 0.6398040 (228)	total: 1m 51s	re

360:	learn: 0.7011925	test: 0.6408030	best: 0.6422168 (313)	total: 2m 22s	remaining: 30m 29s
361:	learn: 0.7008464	test: 0.6398696	best: 0.6422168 (313)	total: 2m 22s	remaining: 30m 28s
362:	learn: 0.7008464	test: 0.6398696	best: 0.6422168 (313)	total: 2m 22s	remaining: 30m 24s
363:	learn: 0.7007978	test: 0.6398696	best: 0.6422168 (313)	total: 2m 23s	remaining: 30m 23s
364:	learn: 0.7008393	test: 0.6400435	best: 0.6422168 (313)	total: 2m 23s	remaining: 30m 22s
365:	learn: 0.7005893	test: 0.6408030	best: 0.6422168 (313)	total: 2m 23s	remaining: 30m 21s
366:	learn: 0.7010753	test: 0.6406080	best: 0.6422168 (313)	total: 2m 24s	remaining: 30m 20s
367:	learn: 0.7010753	test: 0.6406080	best: 0.6422168 (313)	total: 2m 24s	remaining: 30m 18s
368:	learn: 0.7008950	test: 0.6406080	best: 0.6422168 (313)	total: 2m 24s	remaining: 30m 18s
369:	learn: 0.7009437	test: 0.6402389	best: 0.6422168 (313)	total: 2m 25s	remaining: 30m 17s
370:	learn: 0.7013870	test: 0.6400651	best: 0.6422168 (313)	total: 2m 

...

894:	learn: 0.7454170	test: 0.6409079	best: 0.6434971 (433)	total: 5m 49s	remaining: 26m 44s
895:	learn: 0.7454170	test: 0.6409079	best: 0.6434971 (433)	total: 5m 50s	remaining: 26m 43s
896:	learn: 0.7454170	test: 0.6409079	best: 0.6434971 (433)	total: 5m 50s	remaining: 26m 43s
897:	learn: 0.7455385	test: 0.6409079	best: 0.6434971 (433)	total: 5m 51s	remaining: 26m 43s
898:	learn: 0.7454170	test: 0.6401945	best: 0.6434971 (433)	total: 5m 51s	remaining: 26m 43s
899:	learn: 0.7455034	test: 0.6401945	best: 0.6434971 (433)	total: 5m 51s	remaining: 26m 42s
900:	learn: 0.7455034	test: 0.6401945	best: 0.6434971 (433)	total: 5m 52s	remaining: 26m 42s
901:	learn: 0.7461597	test: 0.6412544	best: 0.6434971 (433)	total: 5m 52s	remaining: 26m 41s
902:	learn: 0.7461597	test: 0.6412544	best: 0.6434971 (433)	total: 5m 52s	remaining: 26m 41s
903:	learn: 0.7463324	test: 0.6410603	best: 0.6434971 (433)	total: 5m 53s	remaining: 26m 41s
904:	learn: 0.7462460	test: 0.6412544	best: 0.6434971 (433)	total: 5m 

983:	learn: 0.7516686	test: 0.6385510	best: 0.6434971 (433)	total: 6m 23s	remaining: 26m 4s
984:	learn: 0.7517203	test: 0.6389189	best: 0.6434971 (433)	total: 6m 23s	remaining: 26m 4s
985:	learn: 0.7515310	test: 0.6392867	best: 0.6434971 (433)	total: 6m 24s	remaining: 26m 3s
986:	learn: 0.7516686	test: 0.6385510	best: 0.6434971 (433)	total: 6m 24s	remaining: 26m 3s
987:	learn: 0.7516686	test: 0.6383784	best: 0.6434971 (433)	total: 6m 24s	remaining: 26m 2s
988:	learn: 0.7515827	test: 0.6387463	best: 0.6434971 (433)	total: 6m 25s	remaining: 26m 2s
989:	learn: 0.7515827	test: 0.6387463	best: 0.6434971 (433)	total: 6m 25s	remaining: 26m 1s
990:	learn: 0.7519439	test: 0.6389414	best: 0.6434971 (433)	total: 6m 25s	remaining: 26m 1s
991:	learn: 0.7519439	test: 0.6389414	best: 0.6434971 (433)	total: 6m 26s	remaining: 26m
992:	learn: 0.7521156	test: 0.6389414	best: 0.6434971 (433)	total: 6m 26s	remaining: 25m 59s
993:	learn: 0.7518580	test: 0.6387689	best: 0.6434971 (433)	total: 6m 26s	remainin

1071:	learn: 0.7559868	test: 0.6406250	best: 0.6434971 (433)	total: 6m 54s	remaining: 25m 19s
1072:	learn: 0.7559868	test: 0.6402587	best: 0.6434971 (433)	total: 6m 55s	remaining: 25m 19s
1073:	learn: 0.7559012	test: 0.6402587	best: 0.6434971 (433)	total: 6m 55s	remaining: 25m 18s
1074:	learn: 0.7558492	test: 0.6402587	best: 0.6434971 (433)	total: 6m 55s	remaining: 25m 18s
1075:	learn: 0.7558828	test: 0.6395255	best: 0.6434971 (433)	total: 6m 56s	remaining: 25m 17s
1076:	learn: 0.7560539	test: 0.6400647	best: 0.6434971 (433)	total: 6m 56s	remaining: 25m 17s
1077:	learn: 0.7559684	test: 0.6400647	best: 0.6434971 (433)	total: 6m 56s	remaining: 25m 16s
1078:	learn: 0.7559684	test: 0.6400647	best: 0.6434971 (433)	total: 6m 57s	remaining: 25m 16s
1079:	learn: 0.7559684	test: 0.6400647	best: 0.6434971 (433)	total: 6m 57s	remaining: 25m 15s
1080:	learn: 0.7559684	test: 0.6400647	best: 0.6434971 (433)	total: 6m 57s	remaining: 25m 15s
1081:	learn: 0.7562251	test: 0.6393531	best: 0.6434971 (433)

1159:	learn: 0.7577238	test: 0.6380798	best: 0.6434971 (433)	total: 7m 26s	remaining: 24m 38s
1160:	learn: 0.7577426	test: 0.6375169	best: 0.6434971 (433)	total: 7m 26s	remaining: 24m 37s
1161:	learn: 0.7574861	test: 0.6375169	best: 0.6434971 (433)	total: 7m 27s	remaining: 24m 37s
1162:	learn: 0.7574339	test: 0.6375169	best: 0.6434971 (433)	total: 7m 27s	remaining: 24m 36s
1163:	learn: 0.7574339	test: 0.6375169	best: 0.6434971 (433)	total: 7m 28s	remaining: 24m 36s
1164:	learn: 0.7574339	test: 0.6375169	best: 0.6434971 (433)	total: 7m 28s	remaining: 24m 36s
1165:	learn: 0.7574339	test: 0.6375169	best: 0.6434971 (433)	total: 7m 28s	remaining: 24m 35s
1166:	learn: 0.7573818	test: 0.6375169	best: 0.6434971 (433)	total: 7m 29s	remaining: 24m 34s
1167:	learn: 0.7574339	test: 0.6378845	best: 0.6434971 (433)	total: 7m 29s	remaining: 24m 34s
1168:	learn: 0.7574339	test: 0.6378845	best: 0.6434971 (433)	total: 7m 29s	remaining: 24m 33s
1169:	learn: 0.7571586	test: 0.6378845	best: 0.6434971 (433)

KeyboardInterrupt: 

In [40]:
lr = LogisticRegression()
cross_val_score(lr,train_pred.values[:,:5],train_pred.actual.values,scoring='f1',cv=KFold(5))

C:\Users\asengup6\Softwares\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\asengup6\Softwares\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\asengup6\Softwares\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\asengup6\Softwares\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\asengup6\Softwares\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will 

array([0.65269461, 0.63567568, 0.64026222, 0.64291542, 0.64458322])

In [41]:
lr.fit(train_pred.values[:,:5],train_pred.actual.values)
testpred = lr.predict(test_pred.values)
#testpred = np.round(testpred.as_data_frame()['p1'].values)
submission1 = pd.read_csv('../data/CAX_MortgageModeling_SubmissionFormat.csv')
submission1.Result_Predicted = testpred
submission1.Result_Predicted = submission1.Result_Predicted.apply(lambda x: "FUNDED" if x == 0 else "NOT FUNDED")
print (submission1.Result_Predicted.value_counts(normalize=True))
submission1.to_csv('../submissions/submission21.csv',index=False)

C:\Users\asengup6\Softwares\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


FUNDED        0.667346
NOT FUNDED    0.332654
Name: Result_Predicted, dtype: float64


In [42]:
submission1.head(10)

,Unique_ID,Result_Predicted
0,CAX_Test_1,NOT FUNDED
1,CAX_Test_2,FUNDED
2,CAX_Test_3,FUNDED
3,CAX_Test_4,FUNDED
4,CAX_Test_5,FUNDED
5,CAX_Test_6,NOT FUNDED
6,CAX_Test_7,NOT FUNDED
7,CAX_Test_8,FUNDED
8,CAX_Test_9,FUNDED
9,CAX_Test_10,FUNDED


In [43]:
train_pred.to_csv('../data/blend_train2.csv',index=False)
test_pred.to_csv('../data/blend_test2.csv',index=False)